<a href="https://colab.research.google.com/github/ananthakrishna4747/Anantha_krishna_INFO5731_-Spring2024-/blob/main/In_class_exercise/Chilappagari_krishna_Exercise_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 2**

The purpose of this exercise is to understand users' information needs, and then collect data from different sources for analysis by implementing web scraping using Python.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission. , and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)
Describe an interesting research question (or practical question or something innovative) you have in mind, what kind of data should be collected to answer the question(s)? Specify the amount of data needed for analysis. Provide detailed steps for collecting and saving the data.

In [48]:
# write your answer here

''' I once thought to implement analysis on recent cricket worldcup and i thought if i could predict the match ups
#based on previous stats, how a batsmen performs against a given bowler and vice versa, if it gets implemented
correctly by collecting data, fitting a model, then it would havily impact the betting world or even such apps
like "Dream11", "My11Circle". So i went on to gather data from cricbuzz, use it to fit the model and succeeded
in doing so. This is the idea i had in mind '''

' I once thought to implement analysis on recent cricket worldcup and i thought if i could predict the match ups\n#based on previous stats, how a batsmen performs against a given bowler and vice versa, if it gets implemented\ncorrectly by collecting data, fitting a model, then it would havily impact the betting world or even such apps\nlike "Dream11", "My11Circle". So i went on to gather data from cricbuzz, use it to fit the model and succeeded\nin doing so. This is the idea i had in mind '

## Question 2 (10 Points)
Write Python code to collect a dataset of 1000 samples related to the question discussed in Question 1.

In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

def extract_batting_data(series_id, match_id):

    URL = 'https://www.espncricinfo.com/series/'+ str(series_id) + '/scorecard/' + str(match_id)
    page = requests.get(URL)
    bs = BeautifulSoup(page.content, 'lxml')

    table_body=bs.find_all('tbody')
    batsmen_df = pd.DataFrame(columns=["Name","Desc","Runs", "Balls", "4s", "6s", "SR", "Team"])
    for i, table in enumerate(table_body[0:4:2]):
        rows = table.find_all('tr')
        for row in rows[::2]:
          cols = row.find_all('td')
          if len(cols) < 2:  # Check if cols has at least 2 elements
              continue
          cols = [x.text.strip() for x in cols]
          if cols[0] == 'Extras':
              continue
          if len(cols) > 7:
              new_row = pd.Series(
                  [re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip(), cols[1],
                  cols[2], cols[3], cols[5], cols[6], cols[7], i+1],
                  index=batsmen_df.columns
              )
          else:
              new_row = pd.Series(
                  [re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip(), cols[1],
                  0, 0, 0, 0, 0, i+1],
                  index=batsmen_df.columns
              )
          batsmen_df = pd.concat([batsmen_df, new_row.to_frame().T], ignore_index=True)


    for i in range(min(2, len(bs.find_all("tfoot")))):
        dnb_row = bs.find_all("tfoot")[i].find_all("div")
        for c in dnb_row:
            dnb_cols = c.find_all('span')
            dnb = [x.text.strip().split("(c)")[0] for x in dnb_cols]
            dnb = filter(lambda item: item, [re.sub(r"\W+", ' ', x).strip() for x in dnb])
            for dnb_batsman in dnb:
                batsmen_df = batsmen_df.append(pd.Series([dnb_batsman, "DNB", 0, 0, 0, 0, 0, i+1], index=batsmen_df.columns), ignore_index=True)


    return batsmen_df

# Example usage
extract_batting_data(series_id=8048, match_id=1136561)



,Name,Desc,Runs,Balls,4s,6s,SR,Team
0,Rohit Sharma,c Rayudu b Watson,15,18,1,1,83.33,1
1,Evin Lewis,lbw b Chahar,0,2,0,0,0.00,1
2,Ishan Kishan,c Wood b Imran Tahir,40,29,4,1,137.93,1
3,Suryakumar Yadav,c Harbhajan Singh b Watson,43,29,6,1,148.27,1
4,Hardik Pandya,not out,22,20,2,0,110.00,1
5,Shane Watson,c Lewis b HH Pandya,16,14,1,1,114.28,2
6,Ambati Rayudu,lbw b Markande,22,19,4,0,115.78,2
7,Suresh Raina,c KH Pandya b HH Pandya,4,6,0,0,66.66,2
8,Kedar Jadhav,not out,24,22,1,2,109.09,2


## Question 3 (10 Points)
Write Python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "XYZ". The articles should be published in the last 10 years (2014-2024).

The following information from the article needs to be collected:

(1) Title of the article

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [9]:
import requests
from bs4 import BeautifulSoup

keyword = "NLP"
years_range = range(2014, 2025)

url = f"https://dl.acm.org/action/doSearch?AllField={keyword.lower()}&startPage=&AfterYear={min(years_range)}&BeforeYear={max(years_range) - 1}&queryID=3/6579449608"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
parent_div = soup.find("div", class_="issue-item__content")

articles = soup.select(".issue-item.issue-item--search.clearfix")


for article in articles:
  print("Title : ", article.select_one(".issue-item__title").get_text().strip())
  print("Author : ", article.select_one("ul").get_text().strip()) if (ul := article.select_one("ul")) else None; list_items = [li.get_text() for li in ul.find_all("li")] if ul else None
  print("Citation : ", article.select_one(".issue-item__citation").get_text().strip())
  print("Year : ", article.select_one(".bookPubDate.simple-tooltip__block--b").get_text().strip())
  print("Abstract : ", article.select_one(".issue-item__content-right").get_text().strip())
  print("\n")





#print(articles)


https://dl.acm.org/action/doSearch?AllField=nlp&startPage=&AfterYear=2014&BeforeYear=2023&queryID=3/6579449608
Title :  Natural language processing (NLP) applied on issue trackers
Author :  Mathias Ellmann
Citation :  short-paperNovember 2018Published By ACM
Year :  November 2018
Abstract :  Natural language processing (NLP) applied on issue trackersMathias EllmannNL4SE 2018: Proceedings of the 4th ACM SIGSOFT International Workshop on NLP for Software EngineeringNovember 2018, pp 38–41https://doi.org/10.1145/3283812.3283825
In the domain of software engineering NLP techniques are needed to use and find duplicate or similar development knowledge which are stored in development documentation as development tasks. To understand duplicate and similar development documentations ...6561MetricsTotal Citations6Total Downloads561Last 12 Months78Last 6 weeks4/action/highlight?doi=10.1145%2F3283812.3283825&query=nlpHighlightsExport CitationsSave to BinderSave to Binder
                          

## Question 4A (10 Points)
Develop Python code to collect data from social media platforms like Reddit, Instagram, Twitter (formerly known as X), Facebook, or any other. Use hashtags, keywords, usernames, or user IDs to gather the data.



Ensure that the collected data has more than four columns.


In [ ]:
# write your answer here


## Question 4B (10 Points)
If you encounter challenges with Question-4 web scraping using Python, employ any online tools such as ParseHub or Octoparse for data extraction. Introduce the selected tool, outline the steps for web scraping, and showcase the final output in formats like CSV or Excel.



Upload a document (Word or PDF File) in any shared storage (preferably UNT OneDrive) and add the publicly accessible link in the below code cell.

Please only choose one option for question 4. If you do both options, we will grade only the first one

In [47]:
# write your answer here
"""
Here i've used a tool called Data Scrapper where, we have to use it in the desired website to select all the data needed,
index them, modify according to our needs and then we have to seect no of pages like kind of parsing the next page or dealing
in the same page. when you are done with it, you have to click on start, it  compiles, scraps data from website, and produces
results, where you will have to opt whether to store as excel or csv format. and download the data."""
#the data set downloaded by scraping website ESPNcricinfo stats is stored in below csv file:
# https://drive.google.com/file/d/18Q9ICKVLzBdfRLgGtVm44e4gUJXefmAJ/view?usp=sharing

"\nHere i've used a tool called Data Scrapper where, we have to use it in the desired website to select all the data needed,\nindex them, modify according to our needs and then we have to seect no of pages like kind of parsing the next page or dealing\nin the same page. when you are done with it, you have to click on start, it  compiles, scraps data from website, and produces\nresults, where you will have to opt whether to store as excel or csv format. and download the data."

# Mandatory Question

**Important: Reflective Feedback on Web Scraping and Data Collection**



Please share your thoughts and feedback on the web scraping and data collection exercises you have completed in this assignment. Consider the following points in your response:



Learning Experience: Describe your overall learning experience in working on web scraping tasks. What were the key concepts or techniques you found most beneficial in understanding the process of extracting data from various online sources?



Challenges Encountered: Were there specific difficulties in collecting data from certain websites, and how did you overcome them? If you opted for the non-coding option, share your experience with the chosen tool.



Relevance to Your Field of Study: How might the ability to gather and analyze data from online sources enhance your work or research?

**(no grading of your submission if this question is left unanswered)**

In [7]:
'''
Learning Experience: I found webscapping quite difficult as i faced problems like google blocking me from attempting multiple requests ,
i aslo faced problems while inspecting the page and looking forparticular Element, class ids of the target data.
But it is very much fun using web scrapping tools or extensions like data Scrapper which i used to solve question 4B.
Although i faced problems, i found a way to  overcome them in few instances and left the rest as time's not enough.
I still need to learn, put efforts to learn webscrapping.

Chllenges Encountered: inspecting webpage for particular class_id, getting data from websites is not as easy as i thought,
blockage by google while attempting scrap from google scholar, not enought time for this assignment as scrapping took lot of time.

Relevance to My Field of Study : I've worked on collecting data from websites using automation tools, but never worked on manual scrapping. It is relavent to me and there's need for me to learn this as i may encounter such problems in future as my field includes dealing with such data. So this is very important for me to learn and will focus on this in the coming time. Glad that this is part of course and showed us how important it is to scrap web data
Write your response here.
'''

"\nLearning Experience: I found webscapping quite difficult as i faced problems like google blocking me from attempting multiple requests , \ni aslo faced problems while inspecting the page and looking forparticular Element, class ids of the target data.\nBut it is very much fun using web scrapping tools or extensions like data Scrapper which i used to solve question 4B. \nAlthough i faced problems, i found a way to  overcome them in few instances and left the rest as time's not enough.\nI still need to learn, put efforts to learn webscrapping.\n\nChllenges Encountered: inspecting webpage for particular class_id, getting data from websites is not as easy as i thought, \nblockage by google while attempting scrap from google scholar, not enought time for this assignment as scrapping took lot of time.\n\nRelevance to My Field of Study : I've worked on collecting data from websites using automation tools, but never worked on manual scrapping. It is relavent to me and there's need for me to